In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook
import glob
import re

In [ ]:
# myQLM qpus
from qat.qpus import PyLinalg, CLinalg
qpu_c = CLinalg()
qpu_p = PyLinalg()

In [ ]:
import my_benchmark_execution as bench

In [ ]:
    kernel_configuration = {
        "angles" : ["random", "exact"],
        'auxiliar_qbits_number' : [3, 4, 5],
        "qpu" : "c", #python, qlmass, default
        "fidelity_error" : 0.001,
        "ks_error" : 0.05
    }

    benchmark_arguments = {
        #Pre benchmark sttuff
        "pre_benchmark": True,
        "pre_samples": [10, 20],
        "pre_save": True,
        #Saving stuff
        "save_append" : True,
        "saving_folder": "./Results/",
        "benchmark_times": "kernel_times_benchmark.csv",
        "csv_results": "kernel_benchmark.csv",
        "summary_results": "kernel_SummaryResults.csv",
        #Computing Repetitions stuff
        "alpha": 0.05,
        "min_meas": 15,
        "max_meas": None,
        #List number of qubits tested
        "list_of_qbits": [4, 5],
    }

In [ ]:
    #Configuration for the benchmark kernel
    benchmark_arguments.update({"kernel_configuration": kernel_configuration})
    kernel_bench = bench.KERNEL_BENCHMARK(**benchmark_arguments)
    kernel_bench.exe()

## Analysis of output

In [ ]:
path = './Results/'

In [ ]:
def load_data(complete_folder):
    lista = glob.glob(complete_folder+'*.csv')
    return lista
    print(lista)
    pdf = pd.concat([pd.read_csv(step, index_col=0, header=0) for step in lista])
    pdf.reset_index(drop=True, inplace=True)
    return pdf

In [ ]:
file_list = load_data(path)

In [ ]:
wanted_files = list(filter(
    lambda x: re.search('pre_benchmark', x) is not None, file_list
))

In [ ]:
pdf = pd.concat([pd.read_csv(
    step, index_col=0, header=0, sep =';') for step in wanted_files])

In [ ]:
pdf.groupby(['n_qbits', 'aux_qbits', 'angle_method']).mean()

In [ ]:
pdf.groupby(['n_qbits', 'aux_qbits', 'angle_method']).std()

In [ ]:
file_list

In [ ]:
sumary = pd.read_csv(
    file_list[-1], header=[0, 1], index_col=[0, 1, 2])

In [ ]:
sumary[
    sumary.index.get_level_values('angle_method') == 'exact'
]['KS']

In [ ]:
 "ks_error" : 0.05
        "fidelity_error" : 0.001,

In [ ]:
sumary[
    sumary.index.get_level_values('angle_method') == 'exact'
]['fidelity']

In [ ]:
benchmark = pd.read_csv(file_list [1], sep=';', index_col=0)

In [ ]:
goruped_staf = benchmark.groupby(['n_qbits', 'aux_qbits'])

In [ ]:
goruped_staf.describe()['KS']

In [ ]:
sumary.columns

In [ ]:
sumary2 = sumary.reset_index()

In [ ]:
n_qbits = 5
aux_qbits = 4 
angle_method = 'exact'

In [ ]:
indices = (sumary2['n_qbits'] == n_qbits) & (sumary2['aux_qbits'] == aux_qbits) & (sumary2['angle_method'] == angle_method)

In [ ]:
ks_mean = list(sumary2[indices]['KS']['mean'])[0]
print(ks_mean)
fid_mean = list(sumary2[indices]['fidelity']['mean'])[0]
print(fid_mean)

In [ ]:
from qpe_rz import QPE_RZ

In [ ]:
# Dictionary for configuring the class
qpe_dict = {
    'number_of_qbits' : n_qbits,
    'auxiliar_qbits_number' : aux_qbits,
    'angles' : angle_method,
    #'angles' : 'random',    
    'qpu' : qpu_c,
    'shots' : None
}

In [ ]:
opa=[]
for i in range(50):
    qpe_rz_b = QPE_RZ(**qpe_dict)
    qpe_rz_b.exe()
    opa.append(qpe_rz_b.pdf[['KS', 'fidelity']])

In [ ]:
opa2 =pd.concat(opa).reset_index(drop=True)

In [ ]:
#KS
low = ks_mean - 0.01
high = ks_mean + 0.01

print(sum(opa2['KS'] > high) /len(opa2))
print(sum(opa2['KS'] < low) / len(opa2))

In [ ]:
#fidelity
low = fid_mean - 0.01
high = fid_mean + 0.01

print(sum(opa2['fidelity'] > high) /len(opa2))
print(sum(opa2['fidelity'] < low) / len(opa2))